# Lab 02: Data Exploration

Summary:

1. Data Exploration and initial analysis:
    - Loading the data.
    - Inspecting data shape, column names, and data types, variable types.
    - Viewing basic descriptive statistics and unique values.
    - Subsetting data by rows and columns, and conditional selection.

2. Data Quality Checks:
    - Identifying **missing values**.
    - Checking and correcting **data types**.
    - Detecting and handling **duplicate records**.

4. Data Aggregation and Operations:
    - Summing and aggregating data by columns and rows.
    - Discussing the appropriateness of different operations.

5. Visual Exploration:
    - Plotting univariate distributions (histograms and box plots) for numeric variables.
    - Using matplotlib and seaborn for data visualization.

7. Optional Exercise:
    - Applying the same analysis steps to the project dataset.

In [3]:
# Remember: library imports are ALWAYS at the top of the script, no exceptions!
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'

# Seeting seaborn style
sns.set()

KeyboardInterrupt: 

# Context
The data we will be using through the pratical classes comes from a small relational database whose schema can be seen below:
![alt text](../figures/schema.png "Relation database schema")

# Reading the Data

In [2]:
# path to database
my_path = os.path.join("..", "data", "datamining.db")

# connect to the database
conn = sqlite3.connect(my_path)

# the query
query = """
select
    age, 
    income, 
    frq, 
    rcn, 
    mnt, 
    clothes, 
    kitchen, 
    small_appliances, 
    toys, 
    house_keeping,
    dependents, 
    per_net_purchase,
    g.gender, 
    e.education, 
    m.status, 
    r.description
from customers as c
    join genders as g on g.id = c.gender_id
    join education_levels as e on e.id = c.education_id
    join marital_status as m on m.id = c.marital_status_id
    join recommendations as r on r.id = c.recommendation_id
order by c.id;
"""

df = pd.read_sql_query(query, conn)

NameError: name 'os' is not defined

# Metadata
- *id* - The unique identifier of the customer
- *age* - The year of birht of the customer
- *income* - The income of the customer
- *frq* - Frequency: number of purchases made by the customer
- *rcn* - Recency: number of days since last customer purchase
- *mnt* - Monetary: amount of € spent by the customer in purchases
- *clothes* - Number of clothes items purchased by the customer
- *kitchen* - Number of kitchen items purchased by the customer
- *small_appliances* - Number of small_appliances items purchased by the customer
- *toys* - Number of toys items purchased by the customer
- *house_keeping* - Number of house_keeping items purchased by the customer
- *dependents* - Binary. Whether or not the customer has dependents
- *per_net_purchase* - Percentage of purchases made online
- *education* - Education level of the customer
- *status* - Marital status of the customer
- *gender* - Gender of the customer
- *description* - Last customer's recommendation description

# Initial Analysis

Pandas user guide: https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html

Pandas 10 min tutorial: https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html

## What are some characteristics of our data?

We can use the `.shape` method to see how much data we have.

It returns a tuple with (nrows, ncols).

In [3]:
# Check the shape
df.shape

(8998, 16)

In [4]:
# How to get the names of the columns of the data?
df.columns

Index(['age', 'income', 'frq', 'rcn', 'mnt', 'clothes', 'kitchen',
       'small_appliances', 'toys', 'house_keeping', 'dependents',
       'per_net_purchase', 'gender', 'education', 'status', 'description'],
      dtype='object')

We can use the `.info()` method to get some information about the dataset we have.

In [5]:
# Check the info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8998 entries, 0 to 8997
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   age               8998 non-null   int64 
 1   income            8998 non-null   object
 2   frq               8998 non-null   int64 
 3   rcn               8998 non-null   int64 
 4   mnt               8998 non-null   int64 
 5   clothes           8998 non-null   int64 
 6   kitchen           8998 non-null   int64 
 7   small_appliances  8998 non-null   int64 
 8   toys              8998 non-null   int64 
 9   house_keeping     8998 non-null   int64 
 10  dependents        8998 non-null   object
 11  per_net_purchase  8998 non-null   int64 
 12  gender            8998 non-null   object
 13  education         8998 non-null   object
 14  status            8998 non-null   object
 15  description       8998 non-null   object
dtypes: int64(10), object(6)
memory usage: 1.1+ MB


What do these mean?

- Non-Null Count
- Dtype
    - object
    - float64
    - bool


Let's look at datatypes more closely.

Do their values look correct?

*Hint* compare the dtypes with the provided metadata.

In [8]:
# Check data types


We can use the `.describe()` method to calculate the basic descriptive statistics of our data.

In [ ]:
# Check the descriptive statics
df.describe()
# tentar tirar valores que possam ser interessantes (average)
# tentar encontrar valores inconsistentes 

,age,frq,rcn,mnt,clothes,kitchen,small_appliances,toys,house_keeping,per_net_purchase
count,8998.000000,8998.000000,8998.000000,8998.000000,8998.000000,8998.000000,8998.000000,8998.000000,8998.000000,8998.000000
mean,1966.059680,19.848077,62.469771,622.162814,50.446655,7.039675,28.524116,7.036897,6.929984,42.428984
std,17.296552,10.903435,69.761802,646.768205,23.422249,7.848139,12.586437,7.924422,7.882655,18.495742
min,1936.000000,3.000000,0.000000,6.000000,1.000000,0.000000,1.000000,0.000000,0.000000,4.000000
25%,1951.000000,10.000000,26.000000,63.000000,33.000000,2.000000,19.000000,2.000000,2.000000,28.000000
50%,1966.000000,17.000000,53.000000,383.000000,51.000000,4.000000,28.000000,4.000000,4.000000,45.000000
75%,1981.000000,28.000000,79.000000,1076.000000,69.000000,10.000000,37.000000,10.000000,9.000000,57.000000
max,1996.000000,59.000000,549.000000,3052.000000,99.000000,75.000000,74.000000,62.000000,77.000000,88.000000


By default `.describe()` only considers numeric features.

In [1]:
## Modify the .describe() method to check object types

df.describe(include="object").T

NameError: name 'df' is not defined

In [ ]:
## Modify the .describe() method to check all data types



In [ ]:
# Use these cells to further explore the dataset
# CODE HERE

## Let's do some operations on our data

### Taking a subset of data

You can take a subset with fewer columns by giving a **list** of features:

In [ ]:
## Notice that we use two sets of square brackets

df[['age', 'education']]

You can select a single column of data by providing the name of a single column. This returns a "Series", instead of a DataFrame.

In [ ]:
# How to get the values of one column of data?
# Different ways to access the values of a column


In [ ]:

df['age']

In [ ]:
## Notice the difference

df[['age']]

In [ ]:
# How to get the unique values of one column of data?

df['education'].unique()


#### What about a subset of rows?

In [ ]:
df.loc[[0,1,2,3,4]]

In [ ]:
df.loc[[10,11,12,13,14]]


In [ ]:
## .loc also works for column subsets

df.loc[[0,1,2,3,4], ['age', 'education']]

In [ ]:
## Can you guess what this means?

df.loc[:,:]


#### What about a subset based on a condition?

I want the customers with Age < 40

In [ ]:
df.loc[df['age'] > (2025-40) ]

In [ ]:
## Let's break this down

age = 40
birthyear = (2025-age)
birthyear

In [ ]:
## Let's break this down



In [ ]:
## Let's break this down



In [ ]:
## Let's break this down



### Aggregating data

We can perform mathematical operations on our data

In [ ]:
df['age'].sum()

In [ ]:
## Does it make sense to do this?

df.sum(numeric_only=True)

In [ ]:
## Think of other operations to try: 
## Add two columns together: frq and mnt

## Does this make sense?


You can also do this by row

In [ ]:
## Sum by column

df.sum(numeric_only=True)

In [ ]:
## Sum by row
## Notice the difference

df.sum(numeric_only=True, axis=1)

Discussion:

Does it make sense to add all the numeric features in one row?

## Take a closer look and point out possible problems


In [ ]:
df.describe()


In [ ]:
df.describe(include='object')

In [ ]:
df.head()

### Missing Values

(hint: a missing values in pandas is represented with a NaN value)

We can use the `.isna()` method to see if we have missing values.

In [ ]:
## .isna() returns True if the value in that cell is missing, and False otherwise.

df.isna()

## Is this helpful?

In [ ]:
## .isna() returns True if the value in that cell is missing, and False otherwise.

df.isna().sum()

## What does this mean?
## Is this helpful?

In [ ]:
# count of missing values
df.isna().sum()

In [ ]:
# replace "" by nans
df.replace("", np.nan, inplace=True)

# count of missing values
df.isna().sum()

### Data Types

In [ ]:
# check dataset data types again
df.dtypes

In [ ]:
# fix wrong dtypes
df.dependents = df.dependents.astype("boolean")  # converting to "boolean" over "bool" allows preservation of NaNs

In [ ]:
# check descriptive statistics again
df.describe(include="all").T

### Duplicates

In [ ]:
# duplicated observations
df.duplicated().sum()

*What would it look like if we had duplicates?*

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html


In [ ]:
df_dupes = pd.DataFrame({
     'brand': ['Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
     'style': ['cup', 'cup', 'cup', 'pack', 'pack'],
     'rating': [4, 4, 3.5, 15, 5]
})

df_dupes

In [ ]:
# duplicated observations
df_dupes.duplicated().sum()

In [ ]:
df_dupes.drop_duplicates()

## Take a closer look and point out possible problems:

- Duplicates?
- Data types?
- Missing values?
- Strange values?
- Descriptive statistics?



# Visual Exploration

Matplotlib tutorials: https://matplotlib.org/stable/tutorials/index.html

Matplotlib gallery: https://matplotlib.org/stable/gallery/index.html

Seaborn tutorials: https://seaborn.pydata.org/tutorial.html


Seaborn gallery: https://seaborn.pydata.org/examples/index.html

### Matplotlib vs Seaborn:

**Matplotlib** - lower level. allows to fully customize the plot appearance

**Seaborn** - higher level. Complex off-the-shelf plots with one line. Matplotlib on steroids


In [ ]:
# Define metric and non-metric features. Why?
non_metric_features = ["education", "status", "gender", "dependents", "description"]
metric_features = df.columns.drop(non_metric_features).to_list()

## Pyplot-style vs Object-Oriented-style
- Explicitly create figures and axes, and call methods on them (the "object-oriented (OO) style").
- Rely on pyplot to automatically create and manage the figures and axes, and use pyplot functions for plotting.

More details: https://matplotlib.org/matplotblog/posts/pyplot-vs-object-oriented-interface/

## Numeric Variables' Univariate Distribution

In [ ]:
# Single Metric Variable Histogram
plt.hist(df["age"], bins=10)  # mess around with the bins
plt.title("age", y=-0.2)

plt.show()

In [ ]:
# Single Metric Variable Box Plot
sns.boxplot(y=df["age"])

plt.show()

What information can we extract from the plot above?

In [ ]:
# All Numeric Variables' Histograms in one figure
sns.set()

# Prepare figure. Create individual axes where each histogram will be placed
fig, axes = plt.subplots(2, ceil(len(metric_features) / 2), figsize=(20, 11))

# Plot data
# Iterate across axes objects and associate each histogram (hint: use the ax.hist() instead of plt.hist()):
for ax, feat in zip(axes.flatten(), metric_features): # Notice the zip() function and flatten() method
    # CODE HERE (extra: set each axis title as the feature name)
    
# Layout
# Add a centered title to the figure:
title = "Numeric Variables' Histograms"

plt.suptitle(title)

plt.show()

In [ ]:
# All Numeric Variables' Box Plots in one figure
sns.set()

# Prepare figure. Create individual axes where each box plot will be placed
fig, axes = plt.subplots(2, ceil(len(metric_features) / 2), figsize=(20, 13))

# Plot data
# Iterate across axes objects and associate each box plot (hint: use the ax argument):
for ax, feat in zip(axes.flatten(), metric_features): # Notice the zip() function and flatten() method
    # CODE HERE
    
# Layout
# Add a centered title to the figure:
title = 'Numeric Variables\' Box Plots' # notice the "\"

# CODE HERE
plt.show()

### Insights:
- univariate distributions
- potential univariate outliers

--------------------------------------

### During our Exploratory Data Analysis (EDA), we must also account for:
- Coherence check
- Outliers
- Feature Engineering

### Depending on the context, various steps must be considered when performing Data Preprocessing. 

The most relevant steps are the following:
- Coherence check (find inconsistent values, missing values, outliers and any other problem you may find in your dataset)
- Data editing (fix inconsistent values)
- Data cleansing (drop observations - Outlier removal and removal of inconsistent values and/or features)
- Data wrangling (feature extraction/engineering and transformation)
- Data reduction (reducing the dimensionality of a dataset, producing summary statistics, reducing the number of records in a dataset)

## Optional Exercise

Download the [Amazing Internation Airlines Inc dataset](https://elearning.novaims.unl.pt/mod/folder/view.php?id=96622). Using the  `DM_AIAI_FlightsDB.csv
` and the `DM_AIAI_CustomerDB.csv`  file, perform the same exercises that we did in this notebook.

Identify the metric and non-metric features in this dataset.

Identify if any problems exist:

- Duplicates?
- Data types?
- Missing values?
- Strange values?
- Descriptive statistics?


In [ ]:
# please use pd.read_csv()
# try to note from if the separator is , or ; or \t